In [1]:
from bs4 import BeautifulSoup
import requests

In [80]:
pre = 'https://en.wiktionary.org/wiki/User:Matthias_Buchmeier/pt-en-'
list_alphabet = [str(chr(one)) for one in range(97,123)]
list_alphabet.insert(len(list_alphabet), '0')

In [81]:
def find_pos(s):
  x = ''
  try:   
    x = s.split('{')[1].split('}')[0]
  except:
    x = ''
  return x
def find_see_after(s):
  x = ''
  try:   
    x = s.split('SEE:')[1].strip()
  except:
    x = ''
  return x
def find_sens(s):
  x = ''
  try:   
    x = s.split('(')[1].split(')')[0].strip()
  except:
    x = ''
  return x
def to_line_dict(source, sens, see_after, pos):
  line = '{}#{}#{}#{}'.format(source,sens, see_after, pos)
  return line

In [82]:
to_line_dict('a','b','d','e')

'a#b#d#e'

In [83]:
dict_pos = {
    "prefix":"Prefix",
    "numeral":"numeral",
    "phraseologicalUnit":"phraseologicalUnit",
    "determiner":"determiner",
    "postposition":"postposition",
    "adj":"adjective",
    "v":"verb",
    "pronoun":"Pronoun",
    "interjection":"interjection",
    "conjunction":"conjunction",
    "n":"noun",
    "pn":"pronoun",
    "suffix":"Suffix",
    "proverb":"proverb",
    "adv":"adverb",
    "article":"Article",
    "symbol":"Symbol",
    "particle":"particle",
    "preposition":"preposition",
}

In [84]:
find_after = []
all = []
list_pos = list(dict_pos.keys())
for character in list_alphabet:
  r = requests.get(pre+character)
  soup = BeautifulSoup(r.content)
  list_tr = soup.find_all('tr')
  for tr_tag in list_tr:
    source = ''
    pos = ''
    sens = ''
    targets = []
    try:
      source = tr_tag.td.a.text
      source_sense = tr_tag.td.text
      pos = find_pos(source_sense)
      sens = find_sens(source_sense)
      if pos in list_pos:
        pos = dict_pos[pos]
      if 'SEE' in source_sense:
        see_after = find_see_after(source_sense)
        find_after.append(to_line_dict(source,sens,see_after,pos))
        continue
      len_targets = len(tr_tag.find_all('td')[1].find_all('a'))
      for index in range(len_targets):
        a_text = tr_tag.find_all('td')[1].find_all('a')[index].text
        targets.append(to_line_dict(source,sens,a_text,pos))
      for x in targets:
        all.append(x)
    except:
      continue

In [86]:
print(len(find_after))
list.sort(find_after)
# find_after[0]

0


In [87]:
for i in range(len(find_after)-1):
  if find_after[i]==find_after[i+1]:
    print(i)
    break

In [41]:
find_after[4847:4849]

['mould#mould#mold#noun', 'mould#mould#mold#noun']

In [88]:
import pandas as pd
from io import StringIO

In [89]:
find_after.insert(0,'word#sens#see#pos')
text_find_after = '\n'.join(find_after)
TESTDATA = StringIO(text_find_after)
df1 = pd.read_csv(TESTDATA, error_bad_lines=False, delimiter='#')
df1.head(8)

,word,sens,see,pos


In [90]:
df1  = df1.rename(columns={"word": "word_origin"})
df1

,word_origin,sens,see,pos


In [91]:
all.insert(0,'word#sens#trans#pos')
text_all = '\n'.join(all)
ALL_DATA = StringIO(text_all)
df2 = pd.read_csv(ALL_DATA, error_bad_lines=False,delimiter='#')

b'Skipping line 41784: expected 4 fields, saw 5\n'


In [92]:
df2

,word,sens,trans,pos
0,-a,NaN,feminine,Suffix
1,-a,NaN,third-person,Suffix
2,-a,NaN,singular,Suffix
3,-a,NaN,present,Suffix
4,-a,NaN,indicative,Suffix
...,...,...,...,...
96128,9º,NaN,9th,adjective
96129,+QD+,NaN,bestest,adjective
96130,+QD+,NaN,very,adjective
96131,+QD+,NaN,best,adjective


In [93]:
x = df1.set_index('see').join(df2.set_index('word'),lsuffix='_left', rsuffix='_right')
x

,word_origin,sens_left,pos_left,sens_right,trans,pos_right


In [94]:
x.loc[x['word_origin']=='abecedarium']

,word_origin,sens_left,pos_left,sens_right,trans,pos_right


In [95]:
df_check = x.reset_index()[['word_origin','sens_left','trans','pos_left']].rename(columns={"word_origin": "word","sens_left": "sens","pos_left": "pos"})
df_check = df_check.drop_duplicates().reset_index().drop(columns='index')
df_check

,word,sens,trans,pos


In [96]:
df_dic = df2.append(df_check, ignore_index=True).drop_duplicates(ignore_index=True)

In [97]:
df_dic.to_csv('../EvaluationDictionary/Matthias_Buchmeier_Dic/PT-EN.csv', sep='#',index=False)